In [3]:
from pathlib import Path
import duckdb
import pandas as pd

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")

path_건축_DB_to = data_root / "건축인허가_2025년_02월.db"
path_주택_DB_to = data_root / "주택인허가_2025년_02월.db"

건축허브에서 제공하는 건축인허가, 주택인허가 데이터를 활용하여 분석

2025년 2월 이전 데이터는 위반건축물 등 일부 건축물이 빠져있는 문제가 있어 부득이 2025년 2월 말 데이터로 2015~2024년 인허가 현황을 집계

In [4]:
# Open two DuckDB connections
# Define the database paths
DB_spec = {
    path_건축_DB_to.stem: (
        con_건축 := duckdb.connect(database=path_건축_DB_to, read_only=True)
    ),
    path_주택_DB_to.stem: (
        con_주택 := duckdb.connect(database=path_주택_DB_to, read_only=True)
    ),
}

for db_name, con in DB_spec.items():
    print(f"Database: {db_name}")
    # print the list of tables in the database
    tables = con.execute("SHOW TABLES").fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])
    print()

for db_name, con in DB_spec.items():
    print(f"Database: {db_name}")
    # show columns and heads of the tables
    tables = con.execute("SHOW TABLES").fetchall()
    for table in tables:
        table_name = table[0]
        df = con.sql(f"SELECT * FROM {table_name}").limit(5).df()
        print(f"Columns of {table_name}:")
        print(df.columns)
        print(f"Head of {table_name}:")
        print(df)
        print()

Database: 건축인허가_2025년_02월
Tables in the database:
기본개요
동별개요
층별개요

Database: 주택인허가_2025년_02월
Tables in the database:
기본개요
동별개요
층별개요

Database: 건축인허가_2025년_02월
Columns of 기본개요:
Index(['관리_허가대장_PK', '대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지',
       '특수지_명', '블록', '로트', '지목_코드_명', '지역_코드_명', '지구_코드_명', '구역_코드_명',
       '지목_코드', '지역_코드', '지구_코드', '구역_코드', '건축_구분_코드', '건축_구분_코드_명',
       '대지_면적(㎡)', '건축_면적(㎡)', '건폐_율(%)', '연면적(㎡)', '용적_률_산정_연면적(㎡)',
       '용적_률(%)', '주_건축물_수', '부속_건축물_동_수', '주_용도_코드', '주_용도_코드_명', '세대_수(세대)',
       '호_수(호)', '가구_수(가구)', '총_주차_수', '착공_예정_일', '착공_연기_일', '실제_착공_일',
       '건축_허가_일', '사용승인_일', '생성_일자'],
      dtype='object')
Head of 기본개요:
               관리_허가대장_PK                         대지_위치              건물_명  \
0  1000000000000000045934  충청남도 천안시 동남구 성남면 석곡리 376-8번지              None   
1  1000000000000000001810         인천광역시 중구 운서동 2765-3번지        운서동 2765-3   
2           1063100083397        인천광역시 옹진군 백령면 가을리 16번지              None   
3

In [5]:
# Count the number of records in each table and display them in a DataFrame
# Create a list to store the table names and their record counts
table_counts = []

# Iterate through the DBs and the tables and count the records
for db_name, con in DB_spec.items():
    tables = con.execute("SHOW TABLES").fetchall()
    for table in tables:
        table_name = table[0]
        count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]  # type: ignore
        table_counts.append(
            {"DB name": db_name, "Table Name": table_name, "Record Count": count}
        )

# Convert the list to a DataFrame
record_counts_df = pd.DataFrame(table_counts)

# Display the DataFrame
display(record_counts_df)

,DB name,Table Name,Record Count
0,건축인허가_2025년_02월,기본개요,6207241
1,건축인허가_2025년_02월,동별개요,8060261
2,건축인허가_2025년_02월,층별개요,14361599
3,주택인허가_2025년_02월,기본개요,306182
4,주택인허가_2025년_02월,동별개요,367273
5,주택인허가_2025년_02월,층별개요,3152579


각 DB별, 테이블별 레코드 수 확인

In [6]:
import duckdb
import pandas as pd

# ──────────── 2) 컬럼 정의 ────────────
columns_건축_일자 = [
    "건축_허가_일",
    "착공_예정_일",
    "착공_연기_일",
    "실제_착공_일",
    "사용승인_일",
]

columns_주택_일자 = [
    "승인_일",
    "착공_예정_일",
    "착공_일",
    "사용_검사_예정_일",
    "사용_검사_일",
]


# ──────────── 3) 집계 함수 정의 ────────────
# 2015년부터 2024년까지 YYYYMMDD 형식의 날짜 범위에 해당하는
# 각 컬럼의 건수를 세는 함수 정의
def count_between_dates(
    connection, table_name, column_names, lower="20150101", upper="20241231"
):
    """
    connection : duckdb.Connection
    table_name : str (예: '기본개요')
    column_names : iterable of str
    lower, upper : str (YYYYMMDD 형식)

    반환: pandas.DataFrame with columns ['컬럼명', '조건_이상_이하_건수']
    """
    results = []
    for col in column_names:
        # SQL: 문자열 비교로도 YYYYMMDD 범위 비교가 가능
        sql = f"""
        SELECT
            COUNT(*) AS cnt
        FROM {table_name}
        WHERE "{col}" >= '{lower}'
          AND "{col}" <= '{upper}'
        """
        res = connection.execute(sql).fetchone()  # 집계 결과가 (cnt,) 형태로 돌아옴
        results.append({"컬럼명": col, "레코드 수": res[0]})

    return pd.DataFrame(results)


# ──────────── 4) 건축: 카운트 수행 ────────────
df_건축_결과 = count_between_dates(
    connection=con_건축,
    table_name="기본개요",
    column_names=columns_건축_일자,
    lower="20150101",
    upper="20241231",
)

# ──────────── 5) 주택: 카운트 수행 ────────────
df_주택_결과 = count_between_dates(
    connection=con_주택,
    table_name="기본개요",
    column_names=columns_주택_일자,
    lower="20150101",
    upper="20241231",
)

# ──────────── 6) 결과 출력 ────────────
print("── 건축DB 기본개요: 각 컬럼별 ’20150101‘ 이상, ’20241231‘ 이하 건수 ──")
display(df_건축_결과)

print("\n── 주택DB 기본개요: 각 컬럼별 ’20150101‘ 이상, ’20241231‘ 이하 건수 ──")
display(df_주택_결과)


── 건축DB 기본개요: 각 컬럼별 ’20150101‘ 이상, ’20241231‘ 이하 건수 ──


,컬럼명,레코드 수
0,건축_허가_일,2836765
1,착공_예정_일,1295260
2,착공_연기_일,85951
3,실제_착공_일,1293842
4,사용승인_일,1238884



── 주택DB 기본개요: 각 컬럼별 ’20150101‘ 이상, ’20241231‘ 이하 건수 ──


,컬럼명,레코드 수
0,승인_일,189594
1,착공_예정_일,79073
2,착공_일,37746
3,사용_검사_예정_일,84044
4,사용_검사_일,78184


실제 착공일(건축: 실제 착공 일, 주택: 착공 일) 건수가 착공예정일보다 적음. 준공일(건축: 사용승인 일, 주택: 사용 검사 일)이 있는 경우, 착공을 했다고 보고, 실제 착공일 → 착공 연기일 → 착공 예정일 순으로 착공일 추정

In [7]:
# ─── 건축DB: 일부 컬럼 선택 ───
# 착공일 추정 컬럼 추가: 사용승인일이 있으면 (실제 착공일 → 착공 연기일 → 착공 예정일 → 사용승인일), 없으면 실제 착공일(그대로, null 허용)
sql_건축_기본일부 = """
SELECT
    "관리_허가대장_PK",
    "시군구_코드",
    "건축_구분_코드",
    "건축_구분_코드_명",
    "연면적(㎡)",
    "건축_허가_일",
    "착공_예정_일",
    "착공_연기_일",
    "실제_착공_일",
    "사용승인_일",
    CASE
        WHEN "사용승인_일" IS NOT NULL AND "사용승인_일" != ''
            THEN COALESCE(NULLIF("실제_착공_일", ''), NULLIF("착공_연기_일", ''), NULLIF("착공_예정_일", ''))
        ELSE "실제_착공_일"
    END AS 추정_착공일
FROM 기본개요;
"""

# 쿼리 실행 후 DataFrame으로 가져오기
건축_기본일부 = con_건축.sql(sql_건축_기본일부)

# 결과 확인 (예시)
print(건축_기본일부.limit(5))

# 실제착공일은 NULL이고 사용승인일은 NOT NULL인 데이터 5개를 SQL로 추출
query = """
SELECT *
FROM 건축_기본일부
WHERE ("실제_착공_일" IS NULL OR "실제_착공_일" = '')
    AND ("사용승인_일" IS NOT NULL AND "사용승인_일" != '')
LIMIT 5
"""
result = con_건축.sql(query)
print(result)


┌────────────────────────┬─────────────┬────────────────┬───────────────────┬───────────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┐
│    관리_허가대장_PK    │ 시군구_코드 │ 건축_구분_코드 │ 건축_구분_코드_명 │  연면적(㎡)   │ 건축_허가_일 │ 착공_예정_일 │ 착공_연기_일 │ 실제_착공_일 │ 사용승인_일 │ 추정_착공일 │
│        varchar         │   varchar   │    varchar     │      varchar      │ decimal(18,3) │   varchar    │   varchar    │   varchar    │   varchar    │   varchar   │   varchar   │
├────────────────────────┼─────────────┼────────────────┼───────────────────┼───────────────┼──────────────┼──────────────┼──────────────┼──────────────┼─────────────┼─────────────┤
│ 1000000000000000045934 │ 44131       │ NULL           │ NULL              │        45.000 │ 20220928     │ NULL         │ NULL         │ NULL         │ NULL        │ NULL        │
│ 1000000000000000001810 │ 28110       │ 0100           │ 신축              │       226.100 │ 20220425     │ NULL         │ 20250424     │ NULL       

### 건축행위별 데이터 검증

In [8]:
# 사용승인일이 정상인 데이터를 건축행위별로 세기
query = """
SELECT
    "건축_구분_코드",
    "건축_구분_코드_명",
    COUNT(*) AS count
FROM 건축_기본일부
WHERE "사용승인_일" IS NOT NULL
  AND LENGTH("사용승인_일") = 8
GROUP BY "건축_구분_코드", "건축_구분_코드_명"
ORDER BY "건축_구분_코드"
"""
result = con_건축.sql(query)
print(result)

┌────────────────┬────────────────────┬─────────┐
│ 건축_구분_코드 │ 건축_구분_코드_명  │  count  │
│    varchar     │      varchar       │  int64  │
├────────────────┼────────────────────┼─────────┤
│ %              │ NULL               │       2 │
│ 0              │ NULL               │       1 │
│ 0100           │ 신축               │ 2211648 │
│ 0200           │ 증축               │  775354 │
│ 0210           │ NULL               │      42 │
│ 0300           │ 개축               │    7870 │
│ 0400           │ 재축               │    4307 │
│ 0500           │ 이전               │    2799 │
│ 0600           │ 대수선             │   35179 │
│ 0700           │ 용도변경           │  160908 │
│ 0710           │ NULL               │      11 │
│ 0800           │ 발코니구조변경     │     537 │
│ 0900           │ NULL               │       2 │
│ 1              │ NULL               │       1 │
│ 2000           │ 허가/신고사항변경  │     571 │
│ 3000           │ 가설건축물축조허가 │    8666 │
│ NULL           │ NULL               │    3281 │
├───

In [9]:
# 최근 10년 사용승인 데이터를 건축행위별로 세기
query = """
SELECT
    "건축_구분_코드",
    "건축_구분_코드_명",
    COUNT(*) AS count
FROM 건축_기본일부
WHERE "사용승인_일" IS NOT NULL
    AND "건축_허가_일" >= '20150101'
    AND "건축_허가_일" <= '20241231'
GROUP BY "건축_구분_코드", "건축_구분_코드_명"
ORDER BY "건축_구분_코드"
"""
result = con_건축.sql(query)
print(result)

┌────────────────┬────────────────────┬────────┐
│ 건축_구분_코드 │ 건축_구분_코드_명  │ count  │
│    varchar     │      varchar       │ int64  │
├────────────────┼────────────────────┼────────┤
│ 0100           │ 신축               │ 781765 │
│ 0200           │ 증축               │ 289701 │
│ 0300           │ 개축               │   2563 │
│ 0400           │ 재축               │   1648 │
│ 0500           │ 이전               │    197 │
│ 0600           │ 대수선             │  19469 │
│ 0700           │ 용도변경           │  33864 │
│ 0800           │ 발코니구조변경     │    433 │
│ 2000           │ 허가/신고사항변경  │     49 │
│ 3000           │ 가설건축물축조허가 │   2011 │
│ NULL           │ NULL               │     66 │
├────────────────┴────────────────────┴────────┤
│ 11 rows                            3 columns │
└──────────────────────────────────────────────┘



최근 10년(2015-2024) 건축인허가 데이터에는 건축 구분 코드 오류가 거의 없음(미기재된 경우 66건)

신축, 증축, 개축, 재축, 이전, 대수선만 집계

In [10]:
# 신축+증축/개축/재축/이전+대수선 연계
query = """
SELECT
    *
FROM 건축_기본일부
WHERE "건축_구분_코드" IN ('0100', '0200', '0300', '0400', '0500', '0600')
"""
건축_신축증축대수선 = con_건축.sql(query)

# ─── 2. 동별개요, 층별개요 연계 LEFT JOIN ───
query = """
SELECT 
    a.*,
    b."관리_동별_개요_PK",
    b."연면적(㎡)" AS 동별_연면적,
FROM 건축_신축증축대수선 a
LEFT JOIN 동별개요 b
    ON a."관리_허가대장_PK" = b."관리_허가대장_PK"
"""
건축_join_동별 = con_건축.sql(query)
query = """
SELECT 
    a.*,
    c."관리_층별_개요_PK",
    c."층_구분_코드",
    c."층_구분_코드_명",
    c."주_용도_코드",
    c."주_용도_코드_명",
    c."층_면적(㎡)" AS 층_면적
FROM 건축_join_동별 a
LEFT JOIN 층별개요 c
    ON a."관리_동별_개요_PK" = c."관리_동별_개요_PK"
"""
건축_join_동별층별 = con_건축.sql(query)
print(건축_join_동별층별.limit(5))

┌────────────────────────┬─────────────┬────────────────┬───────────────────┬───────────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┬────────────────────────┬───────────────┬────────────────────────┬──────────────┬─────────────────┬──────────────┬─────────────────┬───────────────┐
│    관리_허가대장_PK    │ 시군구_코드 │ 건축_구분_코드 │ 건축_구분_코드_명 │  연면적(㎡)   │ 건축_허가_일 │ 착공_예정_일 │ 착공_연기_일 │ 실제_착공_일 │ 사용승인_일 │ 추정_착공일 │   관리_동별_개요_PK    │  동별_연면적  │   관리_층별_개요_PK    │ 층_구분_코드 │ 층_구분_코드_명 │ 주_용도_코드 │ 주_용도_코드_명 │    층_면적    │
│        varchar         │   varchar   │    varchar     │      varchar      │ decimal(18,3) │   varchar    │   varchar    │   varchar    │   varchar    │   varchar   │   varchar   │        varchar         │ decimal(18,3) │        varchar         │   varchar    │     varchar     │   varchar    │     varchar     │ decimal(18,3) │
├────────────────────────┼─────────────┼────────────────┼───────────────────┼───────────────┼──────────────┼──

In [11]:
# count the number of records in 건축_신축증축대수선
query = """
SELECT
    COUNT(*) AS count
FROM 건축_신축증축대수선
WHERE "사용승인_일" >= '20150101'
    AND "사용승인_일" <= '20241231'
"""
result = con_건축.sql(query)
print(result)

┌─────────┐
│  count  │
│  int64  │
├─────────┤
│ 1200980 │
└─────────┘



최근 10년 준공, 신축+증축/개축/재축/이전+대수선 1,200,980 건

### 동별개요 검증

In [12]:
# 허가건별 동별 연면적 비교

query = """
WITH summed AS (
    SELECT 
        관리_허가대장_PK,
        건축_구분_코드,
        건축_구분_코드_명,
        MAX("연면적(㎡)") AS 건별_연면적,
        SUM(동별_연면적) AS sum_동별_연면적,
    FROM 건축_join_동별
    GROUP BY 관리_허가대장_PK, "건축_구분_코드", "건축_구분_코드_명"
),
comparison AS (
    SELECT 
        *,
        건별_연면적 = sum_동별_연면적 AS is_equal
    FROM summed
)
SELECT
    건축_구분_코드,
    건축_구분_코드_명,
    COUNT(*) AS count, 
    SUM(CASE WHEN 건별_연면적 IS NULL THEN 1 ELSE 0 END) AS 건별_연면적_null,
    SUM(CASE WHEN sum_동별_연면적 IS NULL THEN 1 ELSE 0 END) AS 동별_연면적_null,
    SUM(CASE WHEN is_equal is NULL THEN 1 ELSE 0 END) AS null_total,
    SUM(CASE WHEN is_equal THEN 1 ELSE 0 END) AS equal,
    SUM(CASE WHEN NOT is_equal THEN 1 ELSE 0 END) AS unequal,
    100.0 * SUM(CASE WHEN is_equal THEN 1 ELSE 0 END) / COUNT(*) AS percentage_equal
FROM comparison
GROUP BY "건축_구분_코드", "건축_구분_코드_명"
ORDER BY "건축_구분_코드"
;
"""
print(con_건축.sql(query))

┌────────────────┬───────────────────┬─────────┬──────────────────┬──────────────────┬────────────┬─────────┬─────────┬────────────────────┐
│ 건축_구분_코드 │ 건축_구분_코드_명 │  count  │ 건별_연면적_null │ 동별_연면적_null │ null_total │  equal  │ unequal │  percentage_equal  │
│    varchar     │      varchar      │  int64  │      int128      │      int128      │   int128   │ int128  │ int128  │       double       │
├────────────────┼───────────────────┼─────────┼──────────────────┼──────────────────┼────────────┼─────────┼─────────┼────────────────────┤
│ 0100           │ 신축              │ 2825419 │                0 │            10502 │      10502 │ 2708289 │  106628 │  95.85442017626413 │
│ 0200           │ 증축              │  918915 │                0 │             4916 │       4916 │   68684 │  845315 │ 7.4744671705217565 │
│ 0300           │ 개축              │   11136 │                0 │              234 │        234 │    7390 │    3512 │  66.36135057471265 │
│ 0400           │ 재축              │    51

신축의 경우 96%에서 일치하지만, 증축의 경우 7%에 그치며, 개축/재축/이전과 대수선의 경우도 일치율이 낮은 편이다. 동별 연면적 집계가 필요하다.

### 층별개요 검증

In [13]:
# 층별개요에서 층_구분_코드, 층_구분_코드_명별 층 면적 집계
query = """
SELECT
    "층_구분_코드",
    "층_구분_코드_명",
    SUM("층_면적(㎡)") AS total_area
FROM 층별개요
GROUP BY "층_구분_코드", "층_구분_코드_명"
ORDER BY "층_구분_코드"
"""
result = con_건축.sql(query)
print(result)

┌──────────────┬─────────────────┬────────────────┐
│ 층_구분_코드 │ 층_구분_코드_명 │   total_area   │
│   varchar    │     varchar     │ decimal(38,3)  │
├──────────────┼─────────────────┼────────────────┤
│ 10           │ 지하            │  375952259.675 │
│ 20           │ 지상            │ 3142221959.357 │
│ 21           │ 복수층(하층)    │        425.860 │
│ 22           │ 복수층(상층)    │        195.260 │
│ 30           │ 옥탑            │   10532106.507 │
│ NULL         │ NULL            │      74160.975 │
└──────────────┴─────────────────┴────────────────┘



옥탑층은 연면적 산정에서 제외됨. 동별 층별 연면적 비교에서 옥탑층은 제외하도록 함.

In [14]:
# 동별 층별 연면적 비교

query = """
WITH summed AS (
    SELECT 
        관리_동별_개요_PK,
        건축_구분_코드,
        건축_구분_코드_명,
        MAX(동별_연면적) AS 동별_연면적,
        SUM(층_면적) AS sum_층_면적
    FROM 건축_join_동별층별
    WHERE "층_구분_코드" NOT IN ('30')  -- 옥탑층 제외
    GROUP BY 관리_동별_개요_PK, "건축_구분_코드", "건축_구분_코드_명"
),
comparison AS (
    SELECT 
        *,
        동별_연면적 = sum_층_면적 AS is_equal
    FROM summed
)
SELECT
    건축_구분_코드,
    건축_구분_코드_명,
    COUNT(*) AS count, 
    SUM(CASE WHEN 동별_연면적 IS NULL THEN 1 ELSE 0 END) AS 동별_연면적_null,
    SUM(CASE WHEN sum_층_면적 IS NULL THEN 1 ELSE 0 END) AS 층별_연면적_null,
    SUM(CASE WHEN is_equal is NULL THEN 1 ELSE 0 END) AS null_total,
    SUM(CASE WHEN is_equal THEN 1 ELSE 0 END) AS equal,
    SUM(CASE WHEN NOT is_equal THEN 1 ELSE 0 END) AS unequal,
    100.0 * SUM(CASE WHEN is_equal THEN 1 ELSE 0 END) / COUNT(*) AS percentage_equal
FROM comparison
GROUP BY "건축_구분_코드", "건축_구분_코드_명"
ORDER BY "건축_구분_코드"
;
"""
print(con_건축.sql(query))

┌────────────────┬───────────────────┬─────────┬──────────────────┬──────────────────┬────────────┬─────────┬─────────┬───────────────────┐
│ 건축_구분_코드 │ 건축_구분_코드_명 │  count  │ 동별_연면적_null │ 층별_연면적_null │ null_total │  equal  │ unequal │ percentage_equal  │
│    varchar     │      varchar      │  int64  │      int128      │      int128      │   int128   │ int128  │ int128  │      double       │
├────────────────┼───────────────────┼─────────┼──────────────────┼──────────────────┼────────────┼─────────┼─────────┼───────────────────┤
│ 0100           │ 신축              │ 3600747 │                0 │                0 │          0 │ 3393404 │  207343 │ 94.24166707630388 │
│ 0200           │ 증축              │ 1406027 │                0 │                0 │          0 │ 1219382 │  186645 │ 86.72536160400902 │
│ 0300           │ 개축              │   15051 │                0 │                0 │          0 │   12888 │    2163 │ 85.62886186964322 │
│ 0400           │ 재축              │    7984 │   

신축과 증축/개축/재축/이전 허가 건에서 동별 연면적과 층별 연면적의 합계는 78%–94% 경우 일치하지만, 일치하지 않는 경우를 무시할 수 없다. 대수선의 경우 일치하는 경우가 40%에 불과하여 층별 면적 집계가 필요하다.

### 극단값 검증

In [15]:
# Query to fetch the 연면적 column
query = """
SELECT "층_면적(㎡)"
FROM 층별개요
WHERE "층_면적(㎡)" IS NOT NULL
"""

# Fetch the data
area_data = con_건축.sql(query).fetchdf()

# Calculate statistics
max_value = area_data["층_면적(㎡)"].max()
mean_value = area_data["층_면적(㎡)"].mean()
percentile_1m = area_data["층_면적(㎡)"].quantile(0.999999)  # 1 - 1/1,000,000
percentile_100k = area_data["층_면적(㎡)"].quantile(0.99999)
percentile_10k = area_data["층_면적(㎡)"].quantile(0.9999)
percentile_1k = area_data["층_면적(㎡)"].quantile(0.999)  # 1 - 1/1,000
percentile_1h = area_data["층_면적(㎡)"].quantile(0.99)  # 1 - 1/100

# Display the results
print(f"Max value:           {max_value: 15.2f}")
print(f"Mean value:          {mean_value: 15.2f}")
print(f"0.999999 percentile: {percentile_1m: 15.2f}")
print(f"0.99999 percentile:  {percentile_100k: 15.2f}")
print(f"0.9999 percentile:   {percentile_10k: 15.2f}")
print(f"0.999 percentile:    {percentile_1k: 15.2f}")
print(f"0.99 percentile:     {percentile_1h: 15.2f}")

Max value:              187102521.00
Mean value:                   245.71
0.999999 percentile:      2083055.28
0.99999 percentile:        153450.00
0.9999 percentile:          31677.67
0.999 percentile:            8891.20
0.99 percentile:             2023.23


In [16]:
# Query to fetch data sorted by 연면적 in descending order
query = """
SELECT *
FROM 층별개요
ORDER BY "층_면적(㎡)" DESC
LIMIT 100
"""

# Execute the query and fetch the result
sorted_data = con_건축.sql(query).fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
    "display.float_format",
    "{:,.2f}".format,  # Format floats as non-scientific with 2 decimal places
):
    print(sorted_data)

               관리_층별_개요_PK             관리_동별_개요_PK              관리_허가대장_PK  \
0                  1234299                 1234232                 1234159   
1                125216954                   12520                   12520   
2   1000000000000001952995  1000000000000001297555  1000000000000000017989   
3   1000000000000001952996  1000000000000001297555  1000000000000000017989   
4           11521000011286          11521000000000          11521000000000   
5   1000000000000000951689  1000000000000000725945  1000000000000000305714   
6   1000000000000000251462  1000000000000000358056  1000000000000000121117   
7                 11756493                   11750                   11750   
8                104921882                10498207                10497249   
9                104921883                10498207                10497249   
10                  120989                  120983                  120957   
11               117112564                11716907              

오류 사례를 거르기 위하여 개별 사례를 검토 후 층별 면적이 500,000 이상인 경우 (0.001% 미만 사례) 제외하도록 하였다. (건축)

In [17]:
# 층별 연면적 집계 (없는 경우 동별, 건별 연면적 사용)
query = """
SELECT 
    관리_허가대장_PK,
    관리_동별_개요_PK,
    시군구_코드,
    주_용도_코드,
    주_용도_코드_명,
    MAX("연면적(㎡)") AS 건별_연면적,
    MAX(동별_연면적) AS 동별_연면적,
    SUM(층_면적) AS 층별_연면적,
    COALESCE(SUM(층_면적), MAX(동별_연면적)) AS 연면적
FROM 건축_join_동별층별
WHERE "층_구분_코드" NOT IN ('30')  -- 옥탑층 제외
    AND 층_면적 <= 500000  -- 층 면적이 500,000㎡ 이하인 경우만
GROUP BY 관리_허가대장_PK, 관리_동별_개요_PK,
    시군구_코드, 주_용도_코드, 주_용도_코드_명
"""
건축_group_동별 = con_건축.sql(query)
print(건축_group_동별.limit(5))
query = """
WITh grouped AS (
    SELECT 
        관리_허가대장_PK,
        시군구_코드,
        주_용도_코드,
        주_용도_코드_명,
        MAX(건별_연면적) AS 건별_연면적,
        SUM(동별_연면적) AS 동별_연면적,
        SUM(층별_연면적) AS 층별_연면적,
        COALESCE(SUM(연면적), MAX(건별_연면적)) AS 연면적
    FROM 건축_group_동별
    GROUP BY 관리_허가대장_PK, 시군구_코드, 주_용도_코드, 주_용도_코드_명
)
SELECT 
    *
FROM grouped
LEFT JOIN 건축_신축증축대수선
    USING (관리_허가대장_PK)
"""
건축_group_건별 = con_건축.sql(query)
print(건축_group_건별.limit(5))


# 최근 10년 건축물 면적 합계 확인
query = """
SELECT 
    건축_구분_코드,
    건축_구분_코드_명,
    SUM(건별_연면적),
    SUM(동별_연면적),
    SUM(층별_연면적),
    SUM(연면적)
FROM 건축_group_건별
WHERE "사용승인_일" >= '20150101'
    AND "사용승인_일" <= '20241231'
GROUP BY 건축_구분_코드, 건축_구분_코드_명
ORDER BY 건축_구분_코드
"""
print(con_건축.sql(query))

┌──────────────────┬───────────────────┬─────────────┬──────────────┬─────────────────┬───────────────┬───────────────┬───────────────┬───────────────┐
│ 관리_허가대장_PK │ 관리_동별_개요_PK │ 시군구_코드 │ 주_용도_코드 │ 주_용도_코드_명 │  건별_연면적  │  동별_연면적  │  층별_연면적  │    연면적     │
│     varchar      │      varchar      │   varchar   │   varchar    │     varchar     │ decimal(18,3) │ decimal(18,3) │ decimal(38,3) │ decimal(38,3) │
├──────────────────┼───────────────────┼─────────────┼──────────────┼─────────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ 10944712         │ 10945114          │ 41171       │ 01003        │ 다가구주택      │       297.080 │       297.080 │       297.080 │       297.080 │
│ 10944739         │ 10945143          │ 41171       │ 01003        │ 다가구주택      │       199.880 │       199.880 │       199.880 │       199.880 │
│ 1099100115027    │ 1099100173178     │ 41220       │ 01003        │ 다가구주택      │       274.290 │       274.290 │       274.290 │       274.290

단순히 층별 면적 집계한 것과 동일한 결과

In [18]:
# 층별개요에서 관리_동별_개요_PK 가 1174100143293 인 데이터 확인
query = """
SELECT *
FROM 층별개요
WHERE "관리_동별_개요_PK" = '1174100143293'
"""
result = con_건축.sql(query)
print(result)

┌───────────────────┬───────────────────┬──────────────────┬─────────────────────────────────┬─────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬───────────┬─────────┬─────────┬───────────┬──────────────┬──────────────┬─────────────────┬─────────┬───────────────┬──────────────┬─────────────────┬────────────────┬───────────────────┬───────────┐
│ 관리_층별_개요_PK │ 관리_동별_개요_PK │ 관리_허가대장_PK │            대지_위치            │ 건물_명 │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │ 특수지_명 │  블록   │  로트   │ 구조_코드 │ 구조_코드_명 │ 주_용도_코드 │ 주_용도_코드_명 │ 층_번호 │  층_면적(㎡)  │ 층_구분_코드 │ 층_구분_코드_명 │ 건축_구분_코드 │ 건축_구분_코드_명 │ 생성_일자 │
│      varchar      │      varchar      │     varchar      │             varchar             │ varchar │   varchar   │   varchar   │    varchar     │ varchar │ varchar │  varchar  │ varchar │ varchar │  varchar  │   varchar    │   varchar    │     varchar     │  int32  │ decimal(18,3) │   varchar    │     varchar     │    varchar     │      varchar      │  varchar

In [19]:
query = """
SELECT 
    시군구_코드,
    주_용도_코드,
    주_용도_코드_명,
    건축_구분_코드,
    건축_구분_코드_명,
    연면적,
    건축_허가_일 AS 허가일,
    추정_착공일 AS 착공일,
    사용승인_일 AS 준공일
FROM 건축_group_건별
"""
final_건축 = con_건축.sql(query)
print(final_건축.limit(5))

┌─────────────┬──────────────┬─────────────────┬────────────────┬───────────────────┬───────────────┬──────────┬──────────┬──────────┐
│ 시군구_코드 │ 주_용도_코드 │ 주_용도_코드_명 │ 건축_구분_코드 │ 건축_구분_코드_명 │    연면적     │  허가일  │  착공일  │  준공일  │
│   varchar   │   varchar    │     varchar     │    varchar     │      varchar      │ decimal(38,3) │ varchar  │ varchar  │ varchar  │
├─────────────┼──────────────┼─────────────────┼────────────────┼───────────────────┼───────────────┼──────────┼──────────┼──────────┤
│ 46130       │ 02003        │ 다세대주택      │ 0100           │ 신축              │       349.670 │ 20191127 │ 20200207 │ 20200807 │
│ 26410       │ 01001        │ 단독주택        │ 0100           │ 신축              │       133.500 │ 20030711 │ 20040610 │ 20040914 │
│ 26200       │ 04499        │ 기타사무소      │ 0100           │ 신축              │        60.700 │ 20121120 │ 20130115 │ 20130604 │
│ 26470       │ 02001        │ 아파트          │ 0100           │ 신축              │      1118.730 │ 20020320 │ 20020511

In [20]:
# ─── 주택DB: 착공일 추정 컬럼 추가 ───
# 사용_검사_일이 있으면 (착공_일 → 착공_예정_일 → 사용_검사_일), 없으면 착공_일(그대로, null 허용)
query = """
SELECT
    "관리_주택대장_PK",
    "시군구_코드",
    "연면적(㎡)",
    "승인_일",
    "착공_예정_일",
    "착공_일",
    "사용_검사_예정_일",
    "사용_검사_일",
    CASE
        WHEN "사용_검사_일" IS NOT NULL AND "사용_검사_일" != ''
            THEN COALESCE(NULLIF("착공_일", ''), NULLIF("착공_예정_일", ''))
        ELSE "착공_일"
    END AS 추정_착공일
FROM 기본개요;
"""
주택_기본일부 = con_주택.sql(query)
print(주택_기본일부.limit(5))

# 착공_일은 NULL이고 추정_착공일은 NOT NULL인 데이터 5개 추출
query = """
SELECT *
FROM 주택_기본일부
WHERE ("착공_일" IS NULL OR "착공_일" = '')
    AND "추정_착공일" IS NOT NULL
LIMIT 5
"""
result = con_주택.sql(query)
print(result)

┌────────────────────────┬─────────────┬───────────────┬──────────┬──────────────┬─────────┬───────────────────┬──────────────┬─────────────┐
│    관리_주택대장_PK    │ 시군구_코드 │  연면적(㎡)   │ 승인_일  │ 착공_예정_일 │ 착공_일 │ 사용_검사_예정_일 │ 사용_검사_일 │ 추정_착공일 │
│        varchar         │   varchar   │ decimal(18,3) │ varchar  │   varchar    │ varchar │      varchar      │   varchar    │   varchar   │
├────────────────────────┼─────────────┼───────────────┼──────────┼──────────────┼─────────┼───────────────────┼──────────────┼─────────────┤
│ 1000000000000000099018 │ 45210       │      7308.110 │ 20230208 │ 20230306     │ NULL    │ 20250317          │ NULL         │ NULL        │
│ 1000000000000000100213 │ 50130       │      2196.150 │ 20221223 │ 20221231     │ NULL    │ 20250114          │ NULL         │ NULL        │
│ 1000000000000000121923 │ 11620       │      1932.110 │ 20201127 │ 20230601     │ NULL    │ 20240909          │ NULL         │ NULL        │
│ 1000000000000000097755 │ 27290       │      498

In [21]:
query = """
SELECT *
FROM 기본개요
WHERE "관리_주택대장_PK" = '1000000000000000011064'
"""
result = con_주택.sql(query)
print(result)
query = """
SELECT *
FROM 기본개요
WHERE "시군구_코드" = '11110'
    AND "법정동_코드" = '17700'
    AND "대지_구분_코드" = '0'
    AND "번" = '0233'
"""
result = con_주택.sql(query)
print(result)
query = """
SELECT *
FROM 동별개요
WHERE "관리_주택대장_PK" = '1000000000000000011064'
"""
result = con_주택.sql(query)
print(result)

┌────────────────────────┬────────────────────────────────┬─────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬───────────┬─────────┬─────────┬───────────┬──────────────┬───────────┬──────────────┬──────────────┬───────────────┬──────────────────┬─────────────────────┬────────────────────────┬──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────┬───────────────────┬──────────────┬───────────┐
│    관리_주택대장_PK    │           대지_위치            │ 건물_명 │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │ 특수지_명 │  블록   │  로트   │ 용도_코드 │ 용도_코드_명 │ 구조_코드 │ 구조_코드_명 │ 주_건축물_수 │  연면적(㎡)   │ 총_세대_수(세대) │ 철거_멸실_구분_코드 │ 철거_멸실_구분_코드_명 │ 철거_시작_일 │ 철거_종료_일 │ 철거_멸실_일 │ 승인_일  │ 착공_예정_일 │ 착공_일 │ 사용_검사_예정_일 │ 사용_검사_일 │ 생성_일자 │
│        varchar         │            varchar             │ varchar │   varchar   │   varchar   │    varchar     │ varchar │ varchar │  varchar  │ varchar │ varchar │  varchar  │   varchar    │  varchar  │   varchar    │    int32     │ dec

경희궁 자이 3단지: 2017년 준공

2022년에 주택법에 따른 행위가 있음

기본개요 연면적을 사용할 수는 없음

검토 결과 신축인 경우만 층별개요에 면적이 부기됨. 층별개요 면적, 주용도코드 집계 가능. 

In [22]:
# 층별개요에서 층_구분_코드, 층_구분_코드_명별 층 면적 집계
query = """
SELECT
    "층_구분_코드",
    "층_구분_코드_명",
    SUM("층_면적(㎡)") AS total_area
FROM 층별개요
GROUP BY "층_구분_코드", "층_구분_코드_명"
ORDER BY "층_구분_코드"
"""
result = con_주택.sql(query)
print(result)

┌──────────────┬─────────────────┬────────────────┐
│ 층_구분_코드 │ 층_구분_코드_명 │   total_area   │
│   varchar    │     varchar     │ decimal(38,3)  │
├──────────────┼─────────────────┼────────────────┤
│ 10           │ 지하            │ 1079829663.535 │
│ 20           │ 지상            │ 2224865241.482 │
│ 21           │ 복수층(하층)    │        446.500 │
│ 23           │ NULL            │        411.880 │
│ 24           │ NULL            │        411.880 │
│ 25           │ NULL            │        411.880 │
│ 26           │ NULL            │        411.880 │
│ 27           │ NULL            │        411.880 │
│ 28           │ NULL            │        411.880 │
│ 29           │ NULL            │        411.880 │
│ 30           │ 옥탑            │     465958.986 │
│ 31           │ NULL            │        411.880 │
│ 32           │ NULL            │        411.880 │
│ 33           │ NULL            │        411.880 │
│ 34           │ NULL            │        411.880 │
│ 35           │ NULL            │

지상, 지하, 복수층(하층), 복수층(상층) (현재는 없음) 해당하는 경우만 집계함.

### 극단값 검증

In [23]:
# Query to fetch the 연면적 column
query = """
SELECT "층_면적(㎡)"
FROM 층별개요
WHERE "층_면적(㎡)" IS NOT NULL
"""

# Fetch the data
area_data = con_주택.sql(query).fetchdf()

# Calculate statistics
max_value = area_data["층_면적(㎡)"].max()
mean_value = area_data["층_면적(㎡)"].mean()
percentile_1m = area_data["층_면적(㎡)"].quantile(0.999999)  # 1 - 1/1,000,000
percentile_100k = area_data["층_면적(㎡)"].quantile(0.99999)
percentile_10k = area_data["층_면적(㎡)"].quantile(0.9999)
percentile_1k = area_data["층_면적(㎡)"].quantile(0.999)  # 1 - 1/1,000
percentile_1h = area_data["층_면적(㎡)"].quantile(0.99)  # 1 - 1/100

# Display the results
print(f"Max value:           {max_value: 15.2f}")
print(f"Mean value:          {mean_value: 15.2f}")
print(f"0.999999 percentile: {percentile_1m: 15.2f}")
print(f"0.99999 percentile:  {percentile_100k: 15.2f}")
print(f"0.9999 percentile:   {percentile_10k: 15.2f}")
print(f"0.999 percentile:    {percentile_1k: 15.2f}")
print(f"0.99 percentile:     {percentile_1h: 15.2f}")

Max value:              218776697.00
Mean value:                  1048.47
0.999999 percentile:     69394550.98
0.99999 percentile:       6302618.00
0.9999 percentile:         420776.00
0.999 percentile:           28255.72
0.99 percentile:             4627.12


In [24]:
# Query to fetch data sorted by 연면적 in descending order
query = """
SELECT *
FROM 층별개요
ORDER BY "층_면적(㎡)" DESC
LIMIT 1000
"""

# Execute the query and fetch the result
sorted_data = con_주택.sql(query).fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
    "display.float_format",
    "{:,.2f}".format,  # Format floats as non-scientific with 2 decimal places
):
    print(sorted_data)

                관리_층별_개요_PK             관리_동별_개요_PK  \
0            10825000072825          10825000009201   
1    1000000000000000608355  1000000000000000148354   
2             1104100053201           1104100012005   
3             1226100033181           1226100006781   
4             1109100016600           1109100007884   
5             1205100026934           1205100008266   
6             1072100007422           1072100004921   
7    1000000000000000071307  1000000000000000021206   
8    1000000000000000071306  1000000000000000021205   
9             1038100028025           1038100007185   
10                 11341206                 1134157   
11                 11341205                 1134157   
12                 10056958                 1005625   
13                 10056957                 1005625   
14            1069100041980           1069100008668   
15            1036100021118           1036100010626   
16            1036100015102           1036100007805   
17        

오류 사례를 거르기 위하여 개별 사례를 검토 후 층별 면적이 500,000 이상인 경우 (0.001% 미만 사례) 제외하도록 하였다. (주택)

In [25]:
# 동별 층별 연계
query = """
SELECT
    a.관리_주택대장_PK,
    a.관리_동별_개요_PK,
    b.관리_층별_개요_PK,
    b.시군구_코드,
    b.층_구분_코드,
    b.층_구분_코드_명,
    b."층_면적(㎡)",
    b.용도_코드 AS 주_용도_코드,
    b.용도_코드_명 AS 주_용도_코드_명
FROM 동별개요 a
JOIN 층별개요 b
    ON a."관리_동별_개요_PK" = b."관리_동별_개요_PK"
WHERE b."층_구분_코드" IN ('10', '20', '21', '22')  -- 지상, 지하, 복수층만
    AND b."층_면적(㎡)" <= 500000  -- 층 면적이 500,000㎡ 이하인 경우만
"""
주택_동별층별 = con_주택.sql(query)
print(주택_동별층별.limit(5))

# 층별 연면적 집계
query = """
SELECT 
    관리_주택대장_PK,
    시군구_코드,
    주_용도_코드,
    주_용도_코드_명,
    SUM("층_면적(㎡)") AS 연면적
FROM 주택_동별층별
GROUP BY 관리_주택대장_PK, 시군구_코드, 주_용도_코드, 주_용도_코드_명
"""
주택_grouped = con_주택.sql(query)
print(주택_grouped.limit(5))

# 인허가 날짜
query = """
SELECT
    관리_주택대장_PK,
    승인_일 AS 허가일,
    추정_착공일 AS 착공일,
    사용_검사_일 AS 준공일
FROM 주택_기본일부
"""
주택_date = con_주택.sql(query)
print(주택_date.limit(5))

query = """
SELECT
    a.관리_주택대장_PK,
    a.시군구_코드,
    a.주_용도_코드,
    a.주_용도_코드_명,
    '0100' AS 건축_구분_코드,
    '신축' AS 건축_구분_코드_명,
    a.연면적,
    b.허가일,
    b.착공일,
    b.준공일
FROM 주택_grouped a
JOIN 주택_date b
    USING (관리_주택대장_PK)
"""
final_주택 = con_주택.sql(query)
print(final_주택.limit(5))

┌──────────────────┬───────────────────┬───────────────────┬─────────────┬──────────────┬─────────────────┬───────────────┬──────────────┬───────────────────────┐
│ 관리_주택대장_PK │ 관리_동별_개요_PK │ 관리_층별_개요_PK │ 시군구_코드 │ 층_구분_코드 │ 층_구분_코드_명 │  층_면적(㎡)  │ 주_용도_코드 │    주_용도_코드_명    │
│     varchar      │      varchar      │      varchar      │   varchar   │   varchar    │     varchar     │ decimal(18,3) │   varchar    │        varchar        │
├──────────────────┼───────────────────┼───────────────────┼─────────────┼──────────────┼─────────────────┼───────────────┼──────────────┼───────────────────────┤
│ 112446           │ 1124719           │ 11246525          │ 41590       │ 20           │ 지상            │       643.508 │ 02001        │ 아파트                │
│ 112446           │ 1124720           │ 11246541          │ 41590       │ 20           │ 지상            │       643.508 │ 02001        │ 아파트                │
│ 112446           │ 1124721           │ 11246543          │ 41590       │ 20    

In [26]:
query = """
SELECT
    LEFT(준공일, 6) AS 준공_년월,
    SUM(연면적)
FROM final_건축
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20240101'
    AND 준공일 <= '20241231'
GROUP BY LEFT(준공일, 6)
ORDER BY LEFT(준공일, 6)
"""
result = con_건축.sql(query)
print(result)

┌───────────┬───────────────┐
│ 준공_년월 │ sum("연면적") │
│  varchar  │ decimal(38,3) │
├───────────┼───────────────┤
│ 202401    │   5979093.873 │
│ 202402    │   5415877.183 │
│ 202403    │   5215905.225 │
│ 202404    │  10501044.972 │
│ 202405    │   4510175.587 │
│ 202406    │   9237905.177 │
│ 202407    │   8020209.298 │
│ 202408    │   6031358.624 │
│ 202409    │   5300345.560 │
│ 202410    │   8497584.895 │
│ 202411    │   5116485.082 │
│ 202412    │   6193995.364 │
├───────────┴───────────────┤
│ 12 rows         2 columns │
└───────────────────────────┘



In [27]:
query = """
SELECT
    LEFT(준공일, 6) AS 준공_년월,
    SUM(연면적)
FROM final_주택
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20240101'
    AND 준공일 <= '20241231'
GROUP BY LEFT(준공일, 6)
ORDER BY LEFT(준공일, 6)
"""
result = con_주택.sql(query)
print(result)

┌───────────┬───────────────┐
│ 준공_년월 │ sum("연면적") │
│  varchar  │ decimal(38,3) │
├───────────┼───────────────┤
│ 202401    │   4607216.398 │
│ 202402    │   3430466.776 │
│ 202403    │   5890011.036 │
│ 202404    │   4740767.679 │
│ 202405    │   3606256.701 │
│ 202406    │   5381213.432 │
│ 202407    │   3434922.443 │
│ 202408    │   5356011.006 │
│ 202409    │   6652007.582 │
│ 202410    │   5525853.437 │
│ 202411    │   5635783.507 │
│ 202412    │   7272491.086 │
├───────────┴───────────────┤
│ 12 rows         2 columns │
└───────────────────────────┘



In [35]:
# 준공일 기준 집계

# 건축
query = """
SELECT
    LEFT(준공일, 6) AS 준공_년월,
    LEFT(시군구_코드, 2) AS 시도_코드,
    LEFT(주_용도_코드, 2) AS 용도_대분류_코드,
    건축_구분_코드,
    건축_구분_코드_명,
    SUM(연면적)
FROM final_건축
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20150101'
    AND 준공일 <= '20241231'
GROUP BY LEFT(준공일, 6), 
    LEFT(시군구_코드, 2),
    LEFT(주_용도_코드, 2),
    건축_구분_코드,
    건축_구분_코드_명,
ORDER BY LEFT(준공일, 6)
"""
df_건축_10년_준공 = con_건축.sql(query).df()
print(df_건축_10년_준공)

# 주택
query = """
SELECT
    LEFT(준공일, 6) AS 준공_년월,
    LEFT(시군구_코드, 2) AS 시도_코드,
    LEFT(주_용도_코드, 2) AS 용도_대분류_코드,
    건축_구분_코드,
    건축_구분_코드_명,
    SUM(연면적)
FROM final_주택
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20150101'
    AND 준공일 <= '20241231'
GROUP BY LEFT(준공일, 6), 
    LEFT(시군구_코드, 2),
    LEFT(주_용도_코드, 2),
    건축_구분_코드,
    건축_구분_코드_명,
ORDER BY LEFT(준공일, 6)
"""
df_주택_10년_준공 = con_주택.sql(query).df()
print(df_주택_10년_준공)

        준공_년월 시도_코드 용도_대분류_코드 건축_구분_코드 건축_구분_코드_명  sum("연면적")
0      201501    41        03     0400         재축      90.000
1      201501    31        04     0100         신축   12459.320
2      201501    26        04     0200         증축    3554.858
3      201501    29        11     0100         신축     500.130
4      201501    28        18     0100         신축    2093.300
...       ...   ...       ...      ...        ...         ...
76978  202412    26        04     0100         신축   15062.912
76979  202412    46        27     0200         증축     222.870
76980  202412    48        18     0200         증축    2936.575
76981  202412    30        06     0200         증축      28.840
76982  202412    31        01     0600        대수선     202.500

[76983 rows x 6 columns]
       준공_년월 시도_코드 용도_대분류_코드 건축_구분_코드 건축_구분_코드_명  sum("연면적")
0     201501    50        03     0100         신축     359.522
1     201501    46        04     0100         신축     569.060
2     201501    11        03     0100         신

In [36]:
# Concatenate the two DataFrames
df_준공_10년_통합 = pd.concat([df_건축_10년_준공, df_주택_10년_준공], ignore_index=True)

# Save to Excel file in 'results' directory
output_path = Path("../results/준공_10년_통합.xlsx")
output_path.parent.mkdir(parents=True, exist_ok=True)
df_준공_10년_통합.to_excel(output_path, index=False)
print(f"Saved to {output_path}")

Saved to ..\results\준공_10년_통합.xlsx


In [40]:
# 준공일 기준 집계

# 건축
query = """
SELECT
    LEFT(준공일, 6) AS 준공_년월,
    LEFT(시군구_코드, 2) AS 시도_코드,
    주_용도_코드,
    건축_구분_코드,
    건축_구분_코드_명,
    SUM(연면적)
FROM final_건축
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20150101'
    AND 준공일 <= '20241231'
    AND 주_용도_코드 <= '02999'  -- 주거용 건축물만
GROUP BY LEFT(준공일, 6), 
    LEFT(시군구_코드, 2),
    주_용도_코드,
    건축_구분_코드,
    건축_구분_코드_명,
ORDER BY LEFT(준공일, 6)
"""
df_건축_10년_준공_주택 = con_건축.sql(query).df()
print(df_건축_10년_준공_주택)

# 주택
query = """
SELECT
    LEFT(준공일, 6) AS 준공_년월,
    LEFT(시군구_코드, 2) AS 시도_코드,
    주_용도_코드,
    건축_구분_코드,
    건축_구분_코드_명,
    SUM(연면적)
FROM final_주택
WHERE 준공일 IS NOT NULL
    AND LENGTH(준공일) = 8
    AND 준공일 >= '20150101'
    AND 준공일 <= '20241231'
    AND 주_용도_코드 <= '02999'  -- 주거용 건축물만
GROUP BY LEFT(준공일, 6), 
    LEFT(시군구_코드, 2),
    주_용도_코드,
    건축_구분_코드,
    건축_구분_코드_명,
ORDER BY LEFT(준공일, 6)
"""
df_주택_10년_준공_주택 = con_주택.sql(query).df()
print(df_주택_10년_준공_주택)

# Concatenate the two DataFrames
df_준공_10년_주택_통합 = pd.concat(
    [df_건축_10년_준공_주택, df_주택_10년_준공_주택], ignore_index=True
)

# Save to Excel file in 'results' directory
output_path = Path("../results/준공_10년_주택_통합.xlsx")
output_path.parent.mkdir(parents=True, exist_ok=True)
df_준공_10년_주택_통합.to_excel(output_path, index=False)
print(f"Saved to {output_path}")

        준공_년월 시도_코드 주_용도_코드 건축_구분_코드 건축_구분_코드_명  sum("연면적")
0      201501    41   02005     0100         신축     561.860
1      201501    29   02003     0100         신축    1753.423
2      201501    48   02001     0100         신축    1263.542
3      201501    11   01002     0200         증축     409.580
4      201501    26   01001     0400         재축      69.550
...       ...   ...     ...      ...        ...         ...
18596  202412    52   02101     0100         신축     195.840
18597  202412    27   01002     0200         증축       0.000
18598  202412    31   01001     0300         개축      37.125
18599  202412    41   01001     0300         개축      19.550
18600  202412    44   01003     0100         신축    2467.090

[18601 rows x 6 columns]
       준공_년월 시도_코드 주_용도_코드 건축_구분_코드 건축_구분_코드_명  sum("연면적")
0     201501    41   02005     0100         신축   57570.330
1     201501    29   02004     0100         신축     470.880
2     201501    48   02001     0100         신축  180369.230
3     201501    26